<a href="https://colab.research.google.com/github/sayakpaul/Training-BatchNorm-and-Only-BatchNorm/blob/master/Varying_Batch_Sizes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Authenticate yourself to use the TPUs
import os

IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [2]:
# TensorFlow Imports
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.58.143.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.58.143.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.58.143.242:8470']
Number of accelerators:  8


In [0]:
!pip install wandb -q
import wandb
wandb.login()

In [0]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/keras-idiomatic-programmer/master/zoo/resnet/resnet_cifar10.py

In [0]:
# Other imports
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import tensorflow as tf
import resnet_cifar10
import numpy as np
import time

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
def get_training_model():
    # ResNet20
    n = 2
    depth =  n * 9 + 2
    n_blocks = ((depth - 2) // 9) - 1

    # The input tensor
    inputs = Input(shape=(32, 32, 3))

    # The Stem Convolution Group
    x = resnet_cifar10.stem(inputs)

    # The learner
    x = resnet_cifar10.learner(x, n_blocks)

    # The Classifier for 10 classes
    outputs = resnet_cifar10.classifier(x, 10)

    # Instantiate the Model
    model = Model(inputs, outputs)

    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            if hasattr(layer, "trainable"):
                layer.trainable=False

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return model

In [0]:
# Load the training set of CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync

def normalize(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.cast(label, tf.int32)
    return image, label

def augment(image,label):
    image = tf.image.resize_with_crop_or_pad(image, 40, 40) # Add 8 pixels of padding
    image = tf.image.random_crop(image, size=[32, 32, 3]) # Random crop back to 32x32
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0., 1.)

    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [10]:
# Train model with with a batch size of 1024
wandb.init(project="training-bn-only", id="bn-bs-{}-crctd-3".format(BATCH_SIZE))

with strategy.scope():
    model = get_training_model()

start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
wandb.log({"batch_size": BATCH_SIZE})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 10s 212ms/step - accuracy: 0.1173 - loss: 2.3456 - val_accuracy: 0.1671 - val_loss: 2.2664
Epoch 2/75
49/49 [==============================] - 3s 68ms/step - accuracy: 0.1490 - loss: 2.2658 - val_accuracy: 0.1579 - val_loss: 2.2433
Epoch 3/75
49/49 [==============================] - 2s 50ms/step - accuracy: 0.1826 - loss: 2.2107 - val_accuracy: 0.1472 - val_loss: 2.2507
Epoch 4/75
49/49 [==============================] - 3s 68ms/step - accuracy: 0.1995 - loss: 2.1722 - val_accuracy: 0.1544 - val_loss: 2.2387
Epoch 5/75
49/49 [==============================] - 3s 70ms/step - accuracy: 0.2093 - loss: 2.1424 - val_accuracy: 0.1667 - val_loss: 2.2020
Epoch 6/75
49/49 [==============================] - 3s 69ms/step - accuracy: 0.2167 - loss: 2.1143 - val_accuracy: 0.1771 - val_loss: 2.1509
Epoch 7/75
49/49 [==============================] - 3s 68ms/step - accuracy: 0.2242 - loss: 2.0915 - val_accuracy: 0.1976 - val_loss: 2.0958
Epoch 8/75


In [11]:
BATCH_SIZE = 32 * strategy.num_replicas_in_sync

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Train model with with a batch size of 256
wandb.init(project="training-bn-only", id="bn-bs-{}-crctd".format(BATCH_SIZE))

with strategy.scope():
    model = get_training_model()

start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
wandb.log({"batch_size": BATCH_SIZE})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
196/196 [==============================] - 14s 73ms/step - accuracy: 0.1413 - loss: 2.3104 - val_accuracy: 0.1195 - val_loss: 2.3373
Epoch 2/75
196/196 [==============================] - 7s 38ms/step - accuracy: 0.1990 - loss: 2.1573 - val_accuracy: 0.2093 - val_loss: 2.0769
Epoch 3/75
196/196 [==============================] - 7s 38ms/step - accuracy: 0.2258 - loss: 2.0839 - val_accuracy: 0.2582 - val_loss: 1.9708
Epoch 4/75
196/196 [==============================] - 7s 38ms/step - accuracy: 0.2423 - loss: 2.0276 - val_accuracy: 0.2830 - val_loss: 1.9150
Epoch 5/75
196/196 [==============================] - 7s 37ms/step - accuracy: 0.2542 - loss: 1.9851 - val_accuracy: 0.2959 - val_loss: 1.8749
Epoch 6/75
196/196 [==============================] - 7s 37ms/step - accuracy: 0.2678 - loss: 1.9473 - val_accuracy: 0.3068 - val_loss: 1.8418
Epoch 7/75
196/196 [==============================] - 7s 37ms/step - accuracy: 0.2756 - loss: 1.9216 - val_accuracy: 0.3144 - val_loss: 1.818

In [12]:
BATCH_SIZE = 64 * strategy.num_replicas_in_sync

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Train model with with a batch size of 512
wandb.init(project="training-bn-only", id="bn-bs-{}".format(BATCH_SIZE))

with strategy.scope():
    model = get_training_model()
    
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
wandb.log({"batch_size": BATCH_SIZE})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
98/98 [==============================] - 13s 128ms/step - accuracy: 0.1247 - loss: 2.3250 - val_accuracy: 0.1369 - val_loss: 2.2511
Epoch 2/75
98/98 [==============================] - 5s 48ms/step - accuracy: 0.1804 - loss: 2.2067 - val_accuracy: 0.1500 - val_loss: 2.2039
Epoch 3/75
98/98 [==============================] - 5s 49ms/step - accuracy: 0.2139 - loss: 2.1329 - val_accuracy: 0.2000 - val_loss: 2.1233
Epoch 4/75
98/98 [==============================] - 5s 49ms/step - accuracy: 0.2286 - loss: 2.0866 - val_accuracy: 0.2191 - val_loss: 2.0634
Epoch 5/75
98/98 [==============================] - 5s 49ms/step - accuracy: 0.2419 - loss: 2.0549 - val_accuracy: 0.2451 - val_loss: 2.0257
Epoch 6/75
98/98 [==============================] - 5s 49ms/step - accuracy: 0.2489 - loss: 2.0307 - val_accuracy: 0.2680 - val_loss: 1.9882
Epoch 7/75
98/98 [==============================] - 5s 50ms/step - accuracy: 0.2560 - loss: 2.0125 - val_accuracy: 0.2801 - val_loss: 1.9637
Epoch 8/75


In [13]:
BATCH_SIZE = 256 * strategy.num_replicas_in_sync

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Train model with with a batch size of 2048
wandb.init(project="training-bn-only", id="bn-bs-{}".format(BATCH_SIZE))

with strategy.scope():
    model = get_training_model()
    
start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
wandb.log({"batch_size": BATCH_SIZE})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
25/25 [==============================] - 10s 419ms/step - accuracy: 0.1122 - loss: 2.3638 - val_accuracy: 0.1204 - val_loss: 2.3491
Epoch 2/75
25/25 [==============================] - 3s 124ms/step - accuracy: 0.1297 - loss: 2.3071 - val_accuracy: 0.1525 - val_loss: 2.2702
Epoch 3/75
25/25 [==============================] - 2s 90ms/step - accuracy: 0.1468 - loss: 2.2638 - val_accuracy: 0.1212 - val_loss: 2.2789
Epoch 4/75
25/25 [==============================] - 2s 88ms/step - accuracy: 0.1635 - loss: 2.2289 - val_accuracy: 0.1125 - val_loss: 2.2900
Epoch 5/75
25/25 [==============================] - 2s 90ms/step - accuracy: 0.1734 - loss: 2.1988 - val_accuracy: 0.1114 - val_loss: 2.3024
Epoch 6/75
25/25 [==============================] - 2s 87ms/step - accuracy: 0.1888 - loss: 2.1718 - val_accuracy: 0.1168 - val_loss: 2.3023
Epoch 7/75
25/25 [==============================] - 2s 91ms/step - accuracy: 0.2003 - loss: 2.1492 - val_accuracy: 0.1270 - val_loss: 2.2892
Epoch 8/75